# Worksheet 08

Name:  Arianit Balidemaj    <br> 
UID: U71435172 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[6.068076840699573, 6.970012474383582, 5.907600563216384, 5.458220974049305, 3.3668203430593415, 5.5992467635311876, 6.171154893227617, 5.354265552419672, 5.892641750915502, 4.53968749248273]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
mean = 8
stdev = 1

c2 = np.random.normal(mean, stdev, 10).tolist()
print(c2)

[8.42473392436914, 7.601097781890937, 8.040357973131915, 9.0459650714404, 7.8246566036057015, 8.671716130981196, 7.737128417608726, 7.910555479131794, 8.160136318963232, 8.8204639128019]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[4.53968749248273, 5.892641750915502, 8.8204639128019, 8.160136318963232, 7.910555479131794, 7.737128417608726, 5.354265552419672, 8.671716130981196, 6.171154893227617, 5.5992467635311876]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The parameters of the GMM that created data are the mean and variance of the two normal distributions that were used to generate the data. The mean of the first normal distribution is 5 and the mean of the second normal distribution is 8. The variance of both normal distributions is 1.


d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.53968749248273, 5.892641750915502, 5.354265552419672, 6.171154893227617, 5.5992467635311876]
[8.8204639128019, 8.160136318963232, 7.910555479131794, 7.737128417608726, 8.671716130981196]
P(C_1) = 0.5,  P(C_2) = 0.5
mean_1 = 5.5113992905153415,  mean_2 = 8.260000051897368
var_1 = 0.31145105550423297,  var_2 = 0.17782177793814113


/Users/arianith.balidemaj/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( prob_c[0] * pdf_i[0] / prob_x )
    prob_c1_x.append( prob_c[1] * pdf_i[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  4.53968749248273
probability of observing that point if it came from cluster 0 =  0.009858017143644676
probability of observing that point if it came from cluster 1 =  2.0084497612366644e-95
point =  5.892641750915502
probability of observing that point if it came from cluster 0 =  0.6055506398718831
probability of observing that point if it came from cluster 1 =  7.312596986125228e-39
point =  8.8204639128019
probability of observing that point if it came from cluster 0 =  3.9369162335608384e-25
probability of observing that point if it came from cluster 1 =  0.015623509031141736
point =  8.160136318963232
probability of observing that point if it came from cluster 0 =  2.523468171871527e-16
probability of observing that point if it came from cluster 1 =  1.91619366446378
point =  7.910555479131794
probability of observing that point if it came from cluster 0 =  1.668410811949879e-13
probability of observing that point if it came from cluster 1 =  0.32535077076505353
point = 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.5000000000350768,  P(C_2) = 0.49999999996492317
mean_1 = 5.511399290671502,  mean_2 = 8.260000051934034
var_1 = 0.3114510558299973,  var_2 = 0.1778217779314519


They are roughly the same in terms of means and variance, very very similar. 

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [9]:
updated_prob_c0_x = [] 
updated_prob_c1_x = []  

for p, pc0_x, pc1_x in zip(data, prob_c0_x, prob_c1_x):
    updated_pc0_x = (pc0_x * prob_c[0]) / prob_x
    updated_pc1_x = (pc1_x * prob_c[1]) / prob_x

    updated_prob_c0_x.append(updated_pc0_x)
    updated_prob_c1_x.append(updated_pc1_x)

print("Updated P(C_0 | X_i) = ", updated_prob_c0_x)
print("Updated P(C_1 | X_i) = ", updated_prob_c1_x)



Updated P(C_0 | X_i) =  [0.8123727470499054, 0.8123727470499054, 2.0470711471976304e-23, 1.0698275486940081e-16, 4.165877558315513e-13, 2.845374935166908e-10, 0.8123727470499054, 2.966665888614295e-22, 0.8123727470499054, 0.8123727470499054]
Updated P(C_1 | X_i) =  [1.6551095680235293e-93, 9.810169636862666e-39, 0.8123727469359234, 0.8123727469359234, 0.8123727469355068, 0.8123727466513859, 1.682755854422356e-58, 0.8123727469359234, 1.4580302189584186e-29, 3.5712690968107795e-49]


It is the exact same thing.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [11]:
hard_assignments = [] 

for pc0_x, pc1_x in zip(updated_prob_c0_x, updated_prob_c1_x):
    if pc0_x >= pc1_x:
        hard_assignments.append(0)
    else:
        hard_assignments.append(1)

print("Hard assignments = ", hard_assignments)

Hard assignments =  [0, 0, 1, 1, 1, 1, 0, 1, 0, 0]


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      Disagree       |
| A  C |      Disagree       |
| A  D |      Disagree       |
| A  E |      Disagree       |
| B  C |      Disagree       |
| B  D |      Disagree       |
| B  E |      Disagree       |
| C  D |      Disagree       |
| C  E |      Disagree       |
| D  E |      Disagree       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

N(N-1)/2

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

36

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

Total Agreements = 3 + 1 + 6 = 10
Total Disagreements = 36 - 10 = 26


e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

We can use a similar algorithm to what we did above but instead of comparing each pair of points we can just compare the points within each cluster. This will reduce the number of comparisons we have to make in general.